<a href="https://colab.research.google.com/github/ZetiAi/library/blob/master/Stable_Worlds_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!pip install -qU diffusers transformers scipy ftfy ipywidgets

In [ ]:
# https://huggingface.co/settings/tokens
!mkdir -p ~/.huggingface
!echo -n "<your-token>" > ~/.huggingface/token

In [ ]:
from tqdm import tqdm, notebook

notebook.tqdm = tqdm

In [ ]:
import torch

torch.set_grad_enabled(False)

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionInpaintPipeline

model_name = "CompVis/stable-diffusion-v1-4"

In [ ]:
generate = StableDiffusionPipeline.from_pretrained(
    model_name, 
    use_auth_token=True,
    torch_dtype=torch.float16, 
    revision="fp16",
).to('cuda')

In [ ]:
inpaint = StableDiffusionInpaintPipeline(
    vae=generate.vae,
    unet=generate.unet,
    scheduler=generate.scheduler,
    tokenizer=generate.tokenizer,
    text_encoder=generate.text_encoder,
    safety_checker=generate.safety_checker,
    feature_extractor=generate.feature_extractor,
).to('cuda')

In [ ]:
# Parameters
num_images = 6
base_width = 768
base_height = 512
num_steps = 100
inpaint_strength = 0.75
prompt = 'a gorgeous street in a futuristic solarpunk city'

In [ ]:
# test if image is good
with torch.autocast('cuda'):
    img = generate(
        prompt=prompt,
        num_inference_steps=num_steps,
        width=base_width,
        height=base_height,
    ).images[0]
display(img)

In [ ]:
# Generate images
imgs = []
with torch.autocast('cuda'):
    for _ in range(num_images):
        img = generate(
            prompt=prompt,
            num_inference_steps=num_steps,
            width=base_width,
            height=base_height,
        ).images[0]
        display(img)
        imgs.append(img)

In [ ]:
# Stitch images
import numpy as np
from PIL import Image, ImageDraw

output = imgs[0]

for img in tqdm(imgs[1:] + [imgs[0]]):
    w,h = output.size
    new = Image.new('RGB', (base_width, base_height))
    new.paste(output, (-w+base_width//2, 0))
    new.paste(img, (base_width//2, 0))

    msk = Image.new('L', (base_width, base_height))
    drw = ImageDraw.Draw(msk)
    drw.rectangle((base_width//4, 0, 3*base_width//4, base_height), fill=255)

    with torch.autocast("cuda"):
        merged = inpaint(
            prompt=prompt,
            init_image=new,
            mask_image=msk,
            strength=inpaint_strength,
            num_inference_steps=num_steps,
        ).images[0]

    new = Image.new('RGB', (w+base_width, base_height))
    new.paste(output, (0, 0))
    new.paste(merged, (w-base_width//2, 0))
    img = img.crop((base_width//2, 0, base_width, base_height))
    new.paste(img, (w+base_width//2, 0))
    output = new

w, h = output.size
output = output.crop((base_width//2, 0, w-base_width//2, h))

In [ ]:
# add borders
w, h = output.size
panorama = Image.new('RGB', (w, 3*h))
panorama.paste(output, (0, h))
display(panorama)

In [ ]:
# save the image and upload here to view:
# https://renderstuff.com/tools/360-panorama-web-viewer/